```bash
QUIZ :  (CLO1 + CLO2 + CLO3 + CLO4) * 5%
UJIAN:  (CLO1 + CLO2 + CLO3 + CLO4) * 12.5%
TUGAS:  (CLO1 + CLO2 + CLO3 + CLO4) * 7.5%

EXAMPLE
QUIZ  => (100 + 100 + 100 + 100) * 5%    = 20
UJIAN => (100 + 100 + 100 + 100) * 12.5% = 50
TUGAS => (100 + 100 + 100 + 100) * 7.5%  = 30

TOTAL NILAI = 100
```

In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
# ======================================
# SISTEM FUZZY METODE MAMDANI
# ======================================

# Definisikan variabel input (0-400)
quiz = ctrl.Antecedent(np.arange(0, 401, 1), 'quiz')
exam = ctrl.Antecedent(np.arange(0, 401, 1), 'exam')
assignment = ctrl.Antecedent(np.arange(0, 401, 1), 'assignment')

# Definisikan variabel output (0-100)
final_score = ctrl.Consequent(np.arange(0, 101, 1), 'final_score')

# Fungsi keanggotaan untuk quiz, exam, assignment (disesuaikan range 0-400)
quiz['low'] = fuzz.trimf(quiz.universe, [0, 0, 200])
quiz['medium'] = fuzz.trimf(quiz.universe, [100, 200, 300])
quiz['high'] = fuzz.trimf(quiz.universe, [200, 400, 400])

exam['low'] = fuzz.trimf(exam.universe, [0, 0, 200])
exam['medium'] = fuzz.trimf(exam.universe, [100, 200, 300])
exam['high'] = fuzz.trimf(exam.universe, [200, 400, 400])

assignment['low'] = fuzz.trimf(assignment.universe, [0, 0, 200])
assignment['medium'] = fuzz.trimf(assignment.universe, [100, 200, 300])
assignment['high'] = fuzz.trimf(assignment.universe, [200, 400, 400])

# Fungsi keanggotaan untuk final_score (0-100)
final_score['F'] = fuzz.trimf(final_score.universe, [0, 0, 50])
final_score['D'] = fuzz.trimf(final_score.universe, [40, 50, 60])
final_score['C'] = fuzz.trimf(final_score.universe, [50, 60, 70])
final_score['B'] = fuzz.trimf(final_score.universe, [65, 75, 85])
final_score['A'] = fuzz.trimf(final_score.universe, [80, 100, 100])

# Aturan fuzzy
rule1 = ctrl.Rule(quiz['high'] & exam['high'] & assignment['high'], final_score['A'])
rule2 = ctrl.Rule(quiz['medium'] & exam['high'] & assignment['medium'], final_score['B'])
rule3 = ctrl.Rule(quiz['medium'] & exam['medium'] & assignment['medium'], final_score['C'])
rule4 = ctrl.Rule(quiz['low'] | exam['low'] | assignment['low'], final_score['F'])
rule5 = ctrl.Rule(quiz['medium'] & exam['low'] & assignment['medium'], final_score['D'])

# Sistem kontrol Mamdani
final_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5])
final_sim = ctrl.ControlSystemSimulation(final_ctrl)

# Contoh input: total nilai 4 CLO (masing-masing 0-100, total 0-400)
nilai_quiz = 300
nilai_exam = 350
nilai_assignment = 320

final_sim.input['quiz'] = nilai_quiz
final_sim.input['exam'] = nilai_exam
final_sim.input['assignment'] = nilai_assignment

final_sim.compute()
nilai_akhir_mamdani = final_sim.output['final_score']
print(f"Nilai akhir (Mamdani) : {nilai_akhir_mamdani:.2f}")

# Output grade Mamdani
if nilai_akhir_mamdani >= 80:
    print("Grade Mamdani: A")
elif nilai_akhir_mamdani >= 70:
    print("Grade Mamdani: AB")
elif nilai_akhir_mamdani >= 65:
    print("Grade Mamdani: B")
elif nilai_akhir_mamdani >= 60:
    print("Grade Mamdani: BC")
elif nilai_akhir_mamdani >= 50:
    print("Grade Mamdani: C")
elif nilai_akhir_mamdani >= 40:
    print("Grade Mamdani: D")
else:
    print("Grade Mamdani: E")

Nilai akhir (Mamdani) : 92.22
Grade Mamdani: A


In [3]:
# ======================================
# SISTEM FUZZY METODE SUGENO
# ======================================

def sugeno_fuzzy(quiz_val, exam_val, assignment_val):
    # Fungsi keanggotaan fuzzy sederhana, returns degree keanggotaan
    def low(x):
        return max(min((200 - x)/200, 1), 0) if x <= 200 else 0
    def medium(x):
        if 100 < x < 200:
            return (x - 100)/100
        elif 200 <= x <= 300:
            return (300 - x)/100
        else:
            return 0
    def high(x):
        return max(min((x - 200)/200, 1), 0) if x >= 200 else 0

    # Derajat keanggotaan tiap input
    quiz_level = {'low': low(quiz_val), 'medium': medium(quiz_val), 'high': high(quiz_val)}
    exam_level = {'low': low(exam_val), 'medium': medium(exam_val), 'high': high(exam_val)}
    assignment_level = {'low': low(assignment_val), 'medium': medium(assignment_val), 'high': high(assignment_val)}

    # Fungsi output linear sesuai bobot
    def weighted_score(q, e, a):
        return (q/400)*20 + (e/400)*50 + (a/400)*30

    # Aturan fuzzy Sugeno
    rules = [
        (min(quiz_level['high'], exam_level['high'], assignment_level['high']), weighted_score),
        (min(quiz_level['medium'], exam_level['high'], assignment_level['medium']), weighted_score),
        (min(quiz_level['medium'], exam_level['medium'], assignment_level['medium']), weighted_score),
        (max(quiz_level['low'], exam_level['low'], assignment_level['low']), weighted_score),
        (min(quiz_level['medium'], exam_level['low'], assignment_level['medium']), weighted_score),
    ]

    numerator = 0
    denominator = 0
    for weight, func in rules:
        if weight > 0:
            f_val = func(quiz_val, exam_val, assignment_val)
            numerator += weight * f_val
            denominator += weight
    
    if denominator == 0:
        return 0
    return numerator / denominator

nilai_akhir_sugeno = sugeno_fuzzy(nilai_quiz, nilai_exam, nilai_assignment)
print(f"Nilai akhir (Sugeno) : {nilai_akhir_sugeno:.2f}")

# Output grade Sugeno
if nilai_akhir_sugeno >= 80:
    print("Grade Sugeno: A")
elif nilai_akhir_sugeno >= 70:
    print("Grade Sugeno: AB")
elif nilai_akhir_sugeno >= 65:
    print("Grade Sugeno: B")
elif nilai_akhir_sugeno >= 60:
    print("Grade Sugeno: BC")
elif nilai_akhir_sugeno >= 50:
    print("Grade Sugeno: C")
elif nilai_akhir_sugeno >= 40:
    print("Grade Sugeno: D")
else:
    print("Grade Sugeno: E")

Nilai akhir (Sugeno) : 82.75
Grade Sugeno: A
